In [75]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from statsmodels.tsa.stattools import adfuller
import datetime
import scipy.stats as stats

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import grid_search_forecaster, backtesting_forecaster
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import Ridge, Lasso

In [110]:
MODEL = 1

In [111]:
index = [0,3] if MODEL==0 else [1,2]
model_name = 'th_v_air' if MODEL==0 else 'el_v_sky'

base_data_train, base_data_test = np.load('../../data/training_data/training_data_1month.npy', allow_pickle=True)

base_data_train, base_data_test = base_data_train[:,:,index], base_data_test[:,:,index]
print(base_data_train.shape, base_data_test.shape)

(108, 730, 2) (12, 730, 2)


Scale base data

In [112]:
from sklearn.preprocessing import MinMaxScaler

scalers = {var_name: MinMaxScaler(feature_range=(-1,1)) for var_name in ['temp', 'energy']}

temp_var, energy_var = base_data_train[:,:,0], base_data_train[:,:,1]
temp_var_test, energy_var_test = base_data_test[:,:,0], base_data_test[:,:,1]

temp_var, temp_var_test = scalers['temp'].fit_transform(temp_var), scalers['temp'].fit_transform(temp_var_test)
energy_var, energy_var_test = scalers['energy'].fit_transform(energy_var), scalers['energy'].fit_transform(energy_var_test)

base_data_train_scaled, base_data_test_scaled = np.stack((temp_var, energy_var), axis=-1), np.stack((temp_var_test, energy_var_test), axis=-1)
print(base_data_train_scaled.shape, base_data_test_scaled.shape)

(108, 730, 2) (12, 730, 2)


PCA?

In [113]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(base_data_train_scaled.reshape(-1,2))
X_test_pca = pca.fit_transform(base_data_test_scaled.reshape(-1,2))

In [114]:
X_train_pca.shape

(78840, 2)

<h2> Use SKforecast to Tune Regression Models</h2>

In [115]:
df = pd.DataFrame(base_data_train_scaled.reshape(-1,2), columns=['temp', 'energy'])
print(df['temp'].index.max())

y_test = pd.Series(base_data_test_scaled[:,:,1].reshape(-1))
exog_var = pd.Series(base_data_test_scaled[:,:,0].reshape(-1))
exog_var.index = exog_var.index + df['temp'].index.max() + 1

78839


<h3>Random Forest</h3>
Tuning Model

In [11]:
forecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123),
                 lags      = 1 # This value will be replaced in the grid search
             )

# Candidate values for lags
lags_grid = [2,6,12,24] 

# Candidate values for regressor's hyperparameters
param_grid = {
    'n_estimators': [2,5,10,50,100],
    'max_depth': [3, 5, 10]
}

results_grid = grid_search_forecaster(
                   forecaster         = forecaster,
                   y                  = df['energy'],
                   exog               = df['temp'],
                   param_grid         = param_grid,
                   lags_grid          = lags_grid,
                   steps              = len(y_test),
                   refit              = False,
                   metric             = 'mean_squared_error',
                   initial_train_size = int(len(base_data_train_scaled)*0.5),
                   fixed_train_size   = False,
                   return_best        = True,
                   n_jobs             = 'auto',
                   verbose            = False,
               )

#results_grid.to_csv(f'../../data/models/model_history/{model_name}_rf_regr_tuning.csv')

NameError: name 'ForecasterAutoreg' is not defined

In [89]:
forecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123),
                 lags      = 1 # This value will be replaced in the grid search
             )

metric, predictions = backtesting_forecaster(
                          forecaster            = forecaster,
                          y                     = df['energy'],
                          steps                 = len(y_test),
                          metric                = 'mean_squared_error',
                          initial_train_size    = int(len(y_test)*0.5),
                          fixed_train_size      = False,
                          gap                   = 0,
                          allow_incomplete_fold = True,
                          refit                 = True,
                          n_jobs                = 'auto',
                          verbose               = True,
                          show_progress         = True  
                      )

Information of backtesting process
----------------------------------
Number of observations used for initial training: 4380
Number of observations used for backtesting: 74460
    Number of folds: 9
    Number of steps per fold: 8760
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 4380 observations.

Fold: 0
    Training:   0 -- 4379  (n=4380)
    Validation: 4380 -- 13139  (n=8760)
Fold: 1
    Training:   0 -- 13139  (n=13140)
    Validation: 13140 -- 21899  (n=8760)
Fold: 2
    Training:   0 -- 21899  (n=21900)
    Validation: 21900 -- 30659  (n=8760)
Fold: 3
    Training:   0 -- 30659  (n=30660)
    Validation: 30660 -- 39419  (n=8760)
Fold: 4
    Training:   0 -- 39419  (n=39420)
    Validation: 39420 -- 48179  (n=8760)
Fold: 5
    Training:   0 -- 48179  (n=48180)
    Validation: 48180 -- 56939  (n=8760)
Fold: 6
    Training:   0 -- 56939  (n=56940)
    Validation: 56940 -- 65699  (n=8760)
Fold: 7
    Training:   0 -- 

  0%|          | 0/9 [00:00<?, ?it/s]

In [90]:
metric

0.3008508329042722

In [ ]:
predictions

In [59]:
results_grid

,lags,params,mean_squared_error,max_depth,n_estimators
0,"[1, 2]","{'max_depth': 3, 'n_estimators': 10}",0.551526,3,10
14,"[1, 2, 3, 4, 5, 6]","{'max_depth': 5, 'n_estimators': 100}",0.558745,5,100
17,"[1, 2, 3, 4, 5, 6]","{'max_depth': 10, 'n_estimators': 100}",0.560708,10,100
26,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","{'max_depth': 10, 'n_estimators': 100}",0.565015,10,100
23,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","{'max_depth': 5, 'n_estimators': 100}",0.567820,5,100
13,"[1, 2, 3, 4, 5, 6]","{'max_depth': 5, 'n_estimators': 50}",0.568645,5,50
16,"[1, 2, 3, 4, 5, 6]","{'max_depth': 10, 'n_estimators': 50}",0.570912,10,50
10,"[1, 2, 3, 4, 5, 6]","{'max_depth': 3, 'n_estimators': 50}",0.571092,3,50
11,"[1, 2, 3, 4, 5, 6]","{'max_depth': 3, 'n_estimators': 100}",0.571838,3,100
20,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","{'max_depth': 3, 'n_estimators': 100}",0.578946,3,100


Select best model, fit to training data and report on test predictions

In [116]:
model_params = {'lag':24, 'max_depth':5, 'n_est':50} if MODEL==0 else {'lag':2, 'max_depth':3, 'n_est':10}
forecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(max_depth=model_params['max_depth'], n_estimators =model_params['n_est']),
                 lags      = model_params['lag']
             )

forecaster.fit(y=df['energy'], exog=df['temp'])

predictions = forecaster.predict(exog=exog_var, steps=len(y_test))

mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2  = r2_score(y_test, predictions)

print(mse, mae, r2)


1.0866935435676843 0.9573950944089874 -0.5911112414434909


<h3> Linear Regression with Ridge regularization </h3>
Tuning Model

In [13]:
forecaster = ForecasterAutoreg(
                 regressor     = Ridge(random_state=123),
                 lags          = 1
             )

param_grid = {'alpha': np.logspace(-5, 5, 10)}
lags_grid = [2, 6, 12, 24]

results_grid = grid_search_forecaster(
                   forecaster         = forecaster,
                   y                  = df['energy'],
                   exog               = df['temp'],
                   param_grid         = param_grid,
                   lags_grid          = lags_grid,
                   steps              = len(y_test),
                   refit              = False,
                   metric             = 'r2',
                   initial_train_size = int(len(df)*0.5),
                   fixed_train_size   = False,
                   return_best        = True,
                   n_jobs             = 'auto',
                   verbose            = False
               )

#results_grid.to_csv(f'../../data/models/model_history/{model_name}_ridge_regr_tuning.csv')

Number of models compared: 40.


lags grid:   0%|          | 0/4 [00:00<?, ?it/s]

params grid:   0%|          | 0/10 [00:00<?, ?it/s]

ValueError: Allowed metrics are: 'mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error' and 'mean_squared_log_error'. Got r2.

In [12]:
results_grid

,lags,params,mean_squared_error,alpha
0,"[1, 2]",{'alpha': 1e-05},0.180834,0.000010
1,"[1, 2]",{'alpha': 0.0001291549665014884},0.180834,0.000129
2,"[1, 2]",{'alpha': 0.0016681005372000592},0.180834,0.001668
3,"[1, 2]",{'alpha': 0.021544346900318846},0.180834,0.021544
4,"[1, 2]",{'alpha': 0.2782559402207126},0.180835,0.278256
5,"[1, 2]",{'alpha': 3.593813663804626},0.180837,3.593814
6,"[1, 2]",{'alpha': 46.41588833612782},0.180865,46.415888
20,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]",{'alpha': 1e-05},0.181030,0.000010
21,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]",{'alpha': 0.0001291549665014884},0.181030,0.000129
22,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]",{'alpha': 0.0016681005372000592},0.181030,0.001668


In [72]:
forecaster = ForecasterAutoreg(
    regressor=Ridge(alpha=0.00010),
    lags = 2 if MODEL==0 else 12
)

forecaster.fit(y=df['energy'], exog=df['temp'])

predictions = forecaster.predict(exog=exog_var, steps=len(y_test))

mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2  = r2_score(y_test, predictions)

print(mse, mae, r2)

0.4410267535978996 0.45813949684691807 -0.0704386341903589


<h3> Gradient Boosting </h3>
Tuning Model

In [100]:
from sklearn.ensemble import GradientBoostingRegressor

forecaster = ForecasterAutoreg(
                 regressor = GradientBoostingRegressor(random_state=123),
                 lags      = 1 # This value will be replaced in the grid search
             )

# Candidate values for lags
lags_grid = [2,6,12,24] 

# Candidate values for regressor's hyperparameters
param_grid = {
    'n_estimators': [2,5,10,50,100],
    'max_depth': [3, 5, 10]
}

results_grid = grid_search_forecaster(
                   forecaster         = forecaster,
                   y                  = df['energy'],
                   exog               = df['temp'],
                   param_grid         = param_grid,
                   lags_grid          = lags_grid,
                   steps              = len(y_test),
                   refit              = False,
                   metric             = 'mean_squared_error',
                   initial_train_size = int(len(base_data_train_scaled)*0.5),
                   fixed_train_size   = False,
                   return_best        = True,
                   n_jobs             = 'auto',
                   verbose            = False
               )

#results_grid.to_csv(f'../../data/models/model_history/{model_name}_gb_regr_tuning.csv')

Number of models compared: 60.


lags grid:   0%|          | 0/4 [00:00<?, ?it/s]

params grid:   0%|          | 0/15 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12] 
  Parameters: {'max_depth': 5, 'n_estimators': 10}
  Backtesting metric: 0.5722609320219454



In [101]:
results_grid.to_csv(f'../../data/models/model_history/{model_name}_gb_regr_tuning.csv')
results_grid

,lags,params,mean_squared_error,max_depth,n_estimators
37,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","{'max_depth': 5, 'n_estimators': 10}",0.572261,5,10
31,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","{'max_depth': 3, 'n_estimators': 5}",0.573512,3,5
32,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","{'max_depth': 3, 'n_estimators': 10}",0.575848,3,10
2,"[1, 2]","{'max_depth': 3, 'n_estimators': 10}",0.578351,3,10
36,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","{'max_depth': 5, 'n_estimators': 5}",0.588277,5,5
41,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","{'max_depth': 10, 'n_estimators': 5}",0.592165,10,5
1,"[1, 2]","{'max_depth': 3, 'n_estimators': 5}",0.596826,3,5
16,"[1, 2, 3, 4, 5, 6]","{'max_depth': 3, 'n_estimators': 5}",0.599768,3,5
7,"[1, 2]","{'max_depth': 5, 'n_estimators': 10}",0.603955,5,10
6,"[1, 2]","{'max_depth': 5, 'n_estimators': 5}",0.606454,5,5


In [109]:
forecaster = ForecasterAutoreg(
    regressor=GradientBoostingRegressor(n_estimators=5, max_depth=10),
    lags = 24
)

forecaster.fit(y=df['energy'], exog=df['temp'])

predictions = forecaster.predict(exog=exog_var, steps=len(y_test))

mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2  = r2_score(y_test, predictions)

print(mse, mae, r2)

0.4461904020072787 0.4552042434660434 -0.08297158985729447
